# Assignment 4: Pipelines and Hyperparameter Tuning (32 total marks)
### Due: November 22 at 11:59pm

### Name: 

### In this assignment, you will be putting together everything you have learned so far. You will need to find your own dataset, do all the appropriate preprocessing, test different supervised learning models and evaluate the results. More details for each step can be found below.

### You will also be asked to describe the process by which you came up with the code. More details can be found below. Please cite any websites or AI tools that you used to help you with this assignment.

## Import Libraries

In [1]:
import numpy as np
import pandas as pd

# Imports for text processing tasks
!pip install contractions > /dev/null
import contractions
import nltk # Imports the Natural Language Toolkit module
import re # regex

# Imports for pipelines, modeling, and evaluation
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score

## Step 1: Data Input (4 marks)

Import the dataset you will be using. You can download the dataset onto your computer and read it in using pandas, or download it directly from the website. Answer the questions below about the dataset you selected. 

To find a dataset, you can use the resources listed in the notes. The dataset can be numerical, categorical, text-based or mixed. If you want help finding a particular dataset related to your interests, please email the instructor.

**You cannot use a dataset that was used for a previous assignment or in class**

In [2]:
# Import dataset (1 mark)
from sklearn.datasets import fetch_20newsgroups

# I selected 3 categories that I like out of the 20 because every run was taking a loooooong time
# Note that because I am still using 3 categories, this is a multiclass classification problem
selected_categories = [
    'sci.electronics',   # Topics related to electronics
    'sci.med',           # Topics related to medical sciences
    'sci.space',         # Topics related to space science 
]

# Fetch only data for the selected topics
newsgroups = fetch_20newsgroups(subset='all', categories=selected_categories)
posts, targets = [s.strip() for s in newsgroups.data], newsgroups.target

### Questions (3 marks)

1. (1 mark) What is the source of your dataset?
1. (1 mark) Why did you pick this particular dataset?
1. (1 mark) Was there anything challenging about finding a dataset that you wanted to use?

*ANSWER HERE*

## Step 2: Data Processing (5 marks)

The next step is to process your data. Implement the following steps as needed.

In [3]:
# Download necessary NLTK resources to process Text Data                   
_ = nltk.download('punkt', quiet=True)           # Downloads a pre-trained tokenizer models used to split sentences
_ = nltk.download('stopwords', quiet=True)       # Downloads a set of stopwords
_ = nltk.download('wordnet', quiet=True)         # Downloads a lexical database of English, used for lemmatization

In [4]:
# Clean data (if needed)

# Custom transformer for word expansion
class WordExpander(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return [contractions.fix(text) for text in X]

# Custom transformer for text cleaning
class TextCleaning(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return [re.sub(r'[^\w\s]', '', text.replace("'s", "")) for text in X]

# Custom transformer for tokenization
class Tokenizer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return [nltk.word_tokenize(text) for text in X]

# Custom transformer for stop word removal
class StopwordRemover(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        stop_words = set(nltk.corpus.stopwords.words('english'))
        return [[word for word in tokens if word.lower() not in stop_words] for tokens in X]

# Custom transformer for lemmatization
class Lemmatizer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        lemmatizer = nltk.stem.WordNetLemmatizer()
        return [' '.join([lemmatizer.lemmatize(word) for word in tokens]) for tokens in X]

In [5]:
# Implement preprocessing steps. Remember to use ColumnTransformer if more than one preprocessing method is needed

### Questions (2 marks)

1. (1 mark) Were there any missing/null values in your dataset? If yes, how did you replace them and why? If no, describe how you would've replaced them and why.
2. (1 mark) What type of data do you have? What preprocessing methods would you have to apply based on your data types?

*ANSWER HERE*

## Step 3: Implement Machine Learning Model (11 marks)

In this section, you will implement three different supervised learning models (one linear and two non-linear) of your choice. You will use a pipeline to help you decide which model and hyperparameters work best. It is up to you to select what models to use and what hyperparameters to test. You can use the class examples for guidance. You must print out the best model parameters and results after the grid search.

In [6]:
# Imports for supervised learning classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

X_train, X_test, y_train, y_test = train_test_split(posts, targets, random_state=0)

In [7]:
# Implement pipeline and grid search here. Can add more code blocks if necessary

# Define scoring metrics for all classifiers
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'f1_score': make_scorer(f1_score, average='weighted')
}

# Create a general pipeline with a placeholder for the classifier
pipeline = Pipeline([
    ('word_expander', WordExpander()),
    ('text_cleaning', TextCleaning()),
    ('tokenizer', Tokenizer()),
    ('stopword_remover', StopwordRemover()),
    ('lemmatizer', Lemmatizer()),
    ('vectorizer', TfidfVectorizer(max_features=5000)),
    ('classifier', None)  # Placeholder for the classifier
])

# Define a combined parameter grid
param_grid = [
    {'classifier': [LogisticRegression(max_iter=5000)],
     'classifier__C': [0.1, 1, 10],
     'classifier__penalty': ['l1', 'l2'],
     'classifier__solver': ['liblinear', 'saga']},
    {'classifier': [MultinomialNB()],
     'classifier__alpha': [0.1, 1, 10]},
    {'classifier': [KNeighborsClassifier()],
     'classifier__n_neighbors': [3, 5, 7],
     'classifier__weights': ['uniform', 'distance']}
]

# Create GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring=scoring, refit='f1_score')

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Print the best model parameters and results
print("Best Overall Estimator:", grid_search.best_estimator_)
print("Best Overall Parameters:", grid_search.best_params_)
print(f'\n Cross-Validation F1 Score: {grid_search.best_score_:.2f}')

Best Overall Estimator: Pipeline(steps=[('word_expander', WordExpander()),
                ('text_cleaning', TextCleaning()), ('tokenizer', Tokenizer()),
                ('stopword_remover', StopwordRemover()),
                ('lemmatizer', Lemmatizer()),
                ('vectorizer', TfidfVectorizer(max_features=5000)),
                ('classifier',
                 LogisticRegression(C=10, max_iter=5000, solver='liblinear'))])
Best Overall Parameters: {'classifier': LogisticRegression(C=10, max_iter=5000, solver='liblinear'), 'classifier__C': 10, 'classifier__penalty': 'l2', 'classifier__solver': 'liblinear'}

 Cross-Validation accuracy 0.98


### Questions (5 marks)

1. (1 mark) Do you need regression or classification models for your dataset?
1. (2 marks) Which models did you select for testing and why?
1. (2 marks) Which model worked the best? Does this make sense based on the theory discussed in the course and the context of your dataset?

*ANSWER HERE*

## Step 4: Validate Model (6 marks)

Use the testing set to calculate the testing accuracy for the best model determined in Step 3.

In [8]:
# Calculate testing accuracy (1 mark)
print(f'Test accuracy {grid_search.score(X_test, y_test):.2f}')

Test accuracy 0.98



### Questions (5 marks)

1. (1 mark) Which accuracy metric did you choose? 
1. (1 mark) How do these results compare to those in part 3? Did this model generalize well?
1. (3 marks) Based on your results and the context of your dataset, did the best model perform "well enough" to be used out in the real-world? Why or why not? Do you have any suggestions for how you could improve this analysis?

*ANSWER HERE*

## Process Description (4 marks)
Please describe the process you used to create your code. Cite any websites or generative AI tools used. You can use the following questions as guidance:
1. Where did you source your code?
1. In what order did you complete the steps?
1. If you used generative AI, what prompts did you use? Did you need to modify the code at all? Why or why not?
1. Did you have any challenges? If yes, what were they? If not, what helped you to be successful?

*DESCRIBE YOUR PROCESS HERE*

## Reflection (2 marks)
Include a sentence or two about:
- what you liked or disliked,
- found interesting, confusing, challenging, motivating
while working on this assignment.


*ADD YOUR THOUGHTS HERE*